In [ ]:
# Import packages
import numpy as np
import matplotlib.pyplot as plt
%matplotlib notebook

from matplotlib.pyplot import figure
from mpl_toolkits.mplot3d import Axes3D
from pprint import pprint

# Setting up random generator
Reproducibility is important when we want to present our works.  
In ML, it is really hard to reproduce results, but we can at least try minimizing randomness by fixing seeds!

In [ ]:
# Set seed for reproducibility.
seed = 0
np.random.seed(0)

# This cell should always output the same value!
print("Test random seed: {}".format(np.random.uniform()))

# 1D Linear Regression
## Generate Training Data
We define the source of training data.  
Realistically, we will not have usually access to real data distribution.  Rather, our job is to learn such a model from training data.

We model the source as a random variable since there is measurement error while observing the data!  
**QUESTION**: What happens if the magnitude of the noise is really high?

**NOTE**: A good practice is to always visualize our data first. This gives us knowledge about what to expect when our trained model does not perform as we might have wanted.

In [ ]:
def generate_data_1d(x, w=np.array([[3], [2]]), noise_mean=0, noise_variance=1):
    """ This is a function for generating a training set (or maybe test set).
    Args:
    - x (ndarray (Shape: (N, 1))): A N-column vector corresponding to the inputs.
    - w (ndarray (Shape: (2, 1))): A 2-column vector corresponding to the parameters of the linear function.
    - noise_mean (float): The mean of a Guassian distribution describing measurement noise.
    - noise_variance (float): The variance of a Guassian distribution describing measurement noise.
    
    Output:
    - y (ndarray (Shape: (N, 1))): A N-column vector corresponding to the outputs given the inputs.
    """
    num_samples = x.shape[0]
    
    # Generate noise
    noise = np.random.normal(loc=noise_mean, scale=noise_variance, size=x.shape)
    
    # Pad with 1's to incorporate the bias term
    x = np.hstack((np.ones((num_samples, 1)), x))
    
    # Compute outputs y, equal to x^T w plus the noise
    y = np.matmul(x, w) + noise
    
    return y

In [ ]:
# Let's generate some training data and visual it

# 1st and 2nd elements of w are offset and slope (here 3, and 2)
true_w = np.array([[3], [2]]) 
noise_mean = 0
noise_variance = 1

# Generate training data, for x in range [0 to 9]
x = np.expand_dims(np.arange(10), axis=1)
y = generate_data_1d(x=x,
                     w=true_w,
                     noise_mean=noise_mean,
                     noise_variance=noise_variance)

# Visualize the dataset
pprint([(input.item(), output.item()) for input, output in zip(x, y)])

fig1 = figure(num=None, figsize=(5, 5), dpi=80, facecolor='w', edgecolor='k')
plt.scatter(x, y)
plt.show()

## Find the optimal parameters, given the training data
We now train our linear model based on the training data generated previously.  

In [ ]:
# Compute the mean/average of x and y.
# Recall that x and y are N-column vectors corresponding to N scalar samples
mean_x = x.mean()
mean_y = y.mean()

In [ ]:
# Compute optimal weight w* given by:
# w* = sum_i((y_i - mean_y) * (x_i - mean_x)) / sum_i((x_i - mean_x)^2)
w = ((y - mean_y) * (x - mean_x)).sum() / ((x - mean_x) ** 2).sum()

In [ ]:
# Compute optimal bias b* given by:
# b* = b* = mean_y - w* * mean_x
b = mean_y - w * mean_x

In [ ]:
# Combine the weight and bias into one vector for convenience
# Also want to check whether our solutions make sense!
w = np.array([[b], [w]])
print(w)

In [ ]:
def visualize_lines(x, observed_y, trained_w, true_w):
    """ This visualizes data with the true and estimated lines """
    fig = figure(num=None, figsize=(10, 10), dpi=80, facecolor='w', edgecolor='k')
    plt.scatter(x, observed_y)
    
    pad_x = np.hstack((np.ones((x.shape[0], 1)), x))
    plt.plot(x, pad_x @ trained_w, label="Estimated Parameters")
    plt.plot(x, pad_x @ true_w, label="Ground Truth Parameters")
    plt.legend()
    plt.show()

visualize_lines(x, y, w, true_w)

We can see that the trained parameters are not the same as the ground truth parameters!  
Why is that?

# 2D Linear Regression
We can modify `generate_data` and generalize to multi-dimensional case.  
For the example below, we will try the 2D case since we can still visualize things!

In [ ]:
def generate_data_nd(x, w, noise_mean=0, noise_variance=1):
    """ This function generates noisy training set (or maybe a test set).
    Args:
    - x (ndarray (Shape: (N, D))): A NxD matrix corresponding to the inputs. Each row correspond to an input
    - w (ndarray (Shape: (D + 1, 1))): A (D + 1)-column vector corresponding to the parameters of the linear function.
    - noise_mean (float): The mean of a Guassian distribution describing measurement noise.
    - noise_variance (float): The variance of a Guassian distribution describing measurement noise.
    
    Output:
    - y (ndarray (Shape: (N, 1))): A N-column vector corresponding to the outputs given the inputs.
    """
    num_samples = x.shape[0]
    
    # Generate noise
    noise = np.random.normal(loc=noise_mean, scale=noise_variance, size=(num_samples, 1))
    
    # Pad 1's for the bias term
    x = np.hstack((np.ones((num_samples, 1)), x))
    # Compute outputs y
    y = np.matmul(x, w) + noise
    
    return y


In [ ]:
# Generate training data 
true_w = np.array([[3], [2], [1]])
noise_mean = 0
noise_variance = 5

# Generate training data
x_1, x_2 = np.meshgrid(np.arange(10), np.arange(10))
x = np.vstack((x_1.flatten(), x_2.flatten())).T
y = generate_data_nd(x=x,
                     w=true_w,
                     noise_mean=noise_mean,
                     noise_variance=noise_variance)

# Visualize the dataset
pprint([(list(input), output.item()) for input, output in zip(x, y)][:20])

fig = figure(num=None, figsize=(8,8), dpi=80, facecolor='w', edgecolor='k')
ax = fig.add_subplot(111, projection='3d')
ax.scatter(x[:, 0], x[:, 1], y)
plt.show()

In [ ]:
def find_optimal_parameters_nd(x, y):
    """ Compute closed form solution for linear regression!
    Optimal weight w* in linear regression is given by w* = (X^T X)^(-1) X^T Y
    
    Args:
    - x (ndarray (Shape: (N, D))): A NxD matrix corresponding to the inputs.
    - y (ndarray (Shape: (N, 1))): A N-column vector corresponding to the outputs given the inputs.
    
    Output:
    - w (ndarray (Shape: (2, 1))): A 2-column vector corresponding to the bias and weight of the linear model.
                                   w = [[bias], [weight]]
    """
    # Pad 1's for the bias term, Why?
    x = np.hstack((np.ones((x.shape[0], 1)), x))

    # Note that we could use pseudoinverse here instead: np.linalg.pinv
    # @ is alias for matmul
    p1 = np.linalg.inv(x.T @ x) # (X^T X) inverse
    p2 = x.T @ y # X^T Y
    w = p1 @ p2
    return w

w = find_optimal_parameters_nd(x, y)
print(w)


In [ ]:
def visualize_planes(x, observed_y, trained_w, true_w, grid_shape=(10, 10)):
    """ This visualize the 3D curves """
    pad_x = np.hstack((np.ones((x.shape[0], 1)), x))
    trained_y = (pad_x @ trained_w).reshape(grid_shape)
    true_y = (pad_x @ true_w).reshape(grid_shape)
    
    fig = figure(num=None, figsize=(10, 10), dpi=80, facecolor='w', edgecolor='k')
    ax = fig.add_subplot(111, projection='3d')
    ax.scatter(x[:, 0], x[:, 1], observed_y)
    x_1 = x[:, 0].reshape(grid_shape)
    x_2 = x[:, 1].reshape(grid_shape)
    ax.plot_surface(x_1, x_2, trained_y, alpha=0.5, color="blue")
    ax.plot_surface(x_1, x_2, true_y, alpha=0.5, color="red")

    plt.show()

visualize_planes(x, y, w, true_w)

Again, as expected, the estimated parameters are close to, but not the same as the ground truth parameters!  